In [1]:
!pip install pandas
!pip install matplotlib


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import pandas as pd

# **1. Робота з архівами даних у Pandas**

## Легенда

Уявімо типовий кейс e-commerce.  

Нам передали архів:

```
ecommerce_events_all.zip
 ├─ 2019-Oct.csv
 ├─ 2019-Nov.csv
 ├─ 2019-Dec.csv
 ├─ 2020-Jan.csv
 └─ 2020-Feb.csv
```

Кожен файл — події користувачів за окремий місяць.  
Дані беремо тут: https://www.kaggle.com/datasets/mkechinov/ecommerce-events-history-in-cosmetics-shop  

Датасет містить поведінкові дані інтернет-магазину косметики за період **жовтень 2019 — лютий 2020**.
Дані надані проєктом **Open CDP**.

**Кожен CSV-файл відповідає одному календарному місяцю спостережень.**

Один файл є **журналом подій користувачів за місяць**.

Файл містить **сирі події**, які фіксують взаємодію користувачів з товарами в інтернет-магазині.

**Кожен рядок — це одна подія.**

---

| Поле            | Тип даних  | Опис                                       | Додаткові примітки                                               |
| --------------- | ---------- | ------------------------------------------ | ---------------------------------------------------------------- |
| `event_time`    | `datetime` | Час, коли відбулася подія                  | Часова зона: **UTC**                                             |
| `event_type`    | `string`   | Тип події користувача                      | Можливі значення: `view`, `cart`, `remove_from_cart`, `purchase` |
| `product_id`    | `int`      | Унікальний ідентифікатор товару            | Один товар може мати багато подій                                |
| `category_id`   | `int`      | Ідентифікатор категорії товару             | Повторюється для різних товарів                                  |
| `category_code` | `string`   | Ієрархічний код категорії товару           | Може бути відсутнім (`NaN`), напр. `electronics.smartphone`      |
| `brand`         | `string`   | Назва бренду товару (lowercase)            | Може бути відсутньою (`NaN`)                                     |
| `price`         | `float`    | Ціна товару на момент події                | Присутня завжди                                                  |
| `user_id`       | `int`      | Постійний ідентифікатор користувача        | Один користувач має багато подій                                 |
| `user_session`  | `string`   | Тимчасовий ідентифікатор сесії користувача | Одна сесія — багато подій; змінюється після довгої паузи         |

---

Поведінкові особливості даних:
* Дані мають **подієву структуру (event-based)**
* Відношення між користувачами і товарами — **many-to-many**
* Один користувач може:
  * мати багато сесій
  * мати багато подій у межах однієї сесії
* Одна сесія може містити кілька подій `purchase`
  (це одна покупка, що складається з кількох товарів)
* Подія `purchase` **не є замовленням** у класичному розумінні. Замовлення потрібно визначати аналітично (наприклад, через агрегацію `purchase` по `user_session`)

## Наївний підхід щодо завантаження даних з архіву (не працює)

```python
pd.read_csv("ecommerce_events.zip")
```

Так можна робити **лише якщо в архіві один CSV**.  
Як тільки файлів більше — Pandas не розуміє, який саме читати.  
**У реальних проєктах цей спосіб майже ніколи не підходить.**

In [3]:
df = pd.read_csv("ecommerce_events_2019_Dec.zip")
df.head()

,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
0,2019-12-01 00:00:00 UTC,remove_from_cart,5712790,1487580005268456287,NaN,f.o.x,6.27,576802932,51d85cb0-897f-48d2-918b-ad63965c12dc
1,2019-12-01 00:00:00 UTC,view,5764655,1487580005411062629,NaN,cnd,29.05,412120092,8adff31e-2051-4894-9758-224bfa8aec18
2,2019-12-01 00:00:02 UTC,cart,4958,1487580009471148064,NaN,runail,1.19,494077766,c99a50e8-2fac-4c4d-89ec-41c05f114554
3,2019-12-01 00:00:05 UTC,view,5848413,1487580007675986893,NaN,freedecor,0.79,348405118,722ffea5-73c0-4924-8e8f-371ff8031af4
4,2019-12-01 00:00:07 UTC,view,5824148,1487580005511725929,NaN,NaN,5.56,576005683,28172809-7e4a-45ce-bab0-5efa90117cd5


In [4]:
df = pd.read_csv("ecommerce_events_all.zip")
df.head()

ValueError: Multiple files found in ZIP file. Only one file per ZIP: ['2019-Dec.csv', '2020-Jan.csv', '2020-Feb.csv', '2019-Oct.csv', '2019-Nov.csv']

## Читання файлів прямо з архіву

1. Відкрити ZIP-архів
2. Перебрати всі CSV
3. Прочитати кожен файл
4. Об’єднати в один DataFrame

### Читаємо окремий файл з архіву

In [5]:
import zipfile

In [15]:
zip_name = 'ecommerce_events_all.zip'
csv_files = ['2019-Dec.csv', '2020-Jan.csv', '2020-Feb.csv', '2019-Oct.csv', '2019-Nov.csv']

dfs = []

In [ ]:
with zipfile.ZipFile(zip_name) as archive:
    for item in csv_files:
        file = archive.open(item)
        df = pd.read_csv(file)
        dfs.append(df)

### Читаємо всі csv-файли з архіву

### Додатковий стовпчик `source_file`

В кожний датасет, який завантажуємо, додаємо стовпчик `source_file`, який дозволяє:

* перевірити, чи всі файли підвантажились
* знайти помилки в конкретному місяці
* зрозуміти, звідки з’явилась аномалія в даних
* швидко відкотитись, якщо один файл виявився «битим»

У реальних проєктах **без цього дебаг перетворюється на гадання**.

Аналітик **ніколи не переходить до аналізу**, доки не переконається, що:

* кількість рядків очікувана
* всі файли враховані
* немає «порожніх» або дивних частин


In [16]:
zip_name = 'ecommerce_events_all.zip'
dfs = []

with zipfile.ZipFile(zip_name) as archive:
    for item in archive.namelist():
        if item.endswith('.csv'):
            file = archive.open(item)
            df = pd.read_csv(file)
            df['source_file'] = item
            dfs.append(df)

In [17]:
len(dfs)

5

In [18]:
df = pd.concat(dfs, ignore_index=True)

In [ ]:
df.head()

In [19]:
df.shape

(20692840, 10)

In [ ]:
df.source_file.unique()

## Робота з типами даних

In [ ]:
df.info()

In [20]:
df['event_time'] = pd.to_datetime(df.event_time)

KeyboardInterrupt: 

In [21]:
df['event_time'].head(5)

0    2019-12-01 00:00:00 UTC
1    2019-12-01 00:00:00 UTC
2    2019-12-01 00:00:02 UTC
3    2019-12-01 00:00:05 UTC
4    2019-12-01 00:00:07 UTC
Name: event_time, dtype: object

In [22]:
df['event_time'] = pd.to_datetime(df.event_time, format='%Y-%m-%d %H:%M:%S UTC', errors='coerce', utc=True)

## Оптимізація памʼяті

Після обʼєднання файлів ми отримали **один великий DataFrame**.
Перше, що потрібно зробити — **дивитися не на графіки, а на памʼять**.

In [23]:
df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20692840 entries, 0 to 20692839
Data columns (total 10 columns):
 #   Column         Dtype              
---  ------         -----              
 0   event_time     datetime64[ns, UTC]
 1   event_type     object             
 2   product_id     int64              
 3   category_id    int64              
 4   category_code  object             
 5   brand          object             
 6   price          float64            
 7   user_id        int64              
 8   user_session   object             
 9   source_file    object             
dtypes: datetime64[ns, UTC](1), float64(1), int64(3), object(5)
memory usage: 6.2 GB


Типова картина:

* багато `object`
* повторювані строкові значення
* зайве споживання памʼяті

> Якщо дані не влазять у памʼять — ніякий аналіз далі не має значення.

### int64 -> int32

In [24]:
-2 ** 31, 2 ** 31 - 1

(-2147483648, 2147483647)

In [25]:
df.product_id.min(), df.product_id.max()

(np.int64(3752), np.int64(5932595))

In [26]:
df.category_id.min(), df.category_id.max()

(np.int64(1487580004807082827), np.int64(2242903426784559183))

In [27]:
df.user_id.min(), df.user_id.max()

(np.int64(465496), np.int64(622090237))

In [28]:
df['product_id'] = df.product_id.astype('int32')
df['user_id'] = df.user_id.astype('int32')

In [29]:
df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20692840 entries, 0 to 20692839
Data columns (total 10 columns):
 #   Column         Dtype              
---  ------         -----              
 0   event_time     datetime64[ns, UTC]
 1   event_type     object             
 2   product_id     int32              
 3   category_id    int64              
 4   category_code  object             
 5   brand          object             
 6   price          float64            
 7   user_id        int32              
 8   user_session   object             
 9   source_file    object             
dtypes: datetime64[ns, UTC](1), float64(1), int32(2), int64(1), object(5)
memory usage: 6.0 GB


### str -> category

In [30]:
df.event_type.unique()

array(['remove_from_cart', 'view', 'cart', 'purchase'], dtype=object)

In [31]:
df['event_type'] = df.event_type.astype('category')

In [32]:
df.category_code.unique()

array([nan, 'furniture.bathroom.bath', 'appliances.environment.vacuum',
       'stationery.cartrige', 'furniture.living_room.cabinet',
       'apparel.glove', 'accessories.bag', 'accessories.cosmetic_bag',
       'appliances.personal.hair_cutter', 'furniture.living_room.chair',
       'appliances.environment.air_conditioner',
       'appliances.personal.massager', 'sport.diving'], dtype=object)

In [33]:
df.brand.unique()

array(['f.o.x', 'cnd', 'runail', 'freedecor', nan, 'ingarden', 'roubloff',
       'metzger', 'staleks', 'zinger', 'yoko', 'entity', 'irisk',
       'beautix', 'uno', 'philips', 'domix', 'masura', 'grattol',
       'strong', 'skinlite', 'nagaraku', 'italwax', 'max', 'bluesky',
       'pole', 'cosmoprofi', 'severina', 'bpw.style', 'igrobeauty',
       'smart', 'coifin', 'estel', 'lador', 'kapous', 'sophin',
       'marathon', 'nitrimax', 'jessnail', 'jas', 'de.lux', 'haruyama',
       'orly', 'lianail', 'almea', 'milv', 'lakme', 'concept', 'pnb',
       'bodyton', 'rosi', 'petitfee', 'polarus', 'lovely', 'oniq',
       'provoc', 'solomeya', 'shik', 'bioaqua', 'art-visage',
       'refectocil', 'freshbubble', 'elskin', 'swarovski', 'airnails',
       'kosmekka', 'bergamo', 'keen', 'gehwol', 'kinetics', 'estelare',
       'missha', 'candy', 'naomi', 'thuya', 'balbcare', 'uskusi',
       'farmavita', 'insight', 'ardell', 'limoni', 'happyfons',
       'bespecial', 'beauty-free', 'carmex', 'b

In [34]:
df['brand'] = df.brand.astype('category')
df['category_code'] = df.category_code.astype('category')

In [35]:
df['source_file'] = df.source_file.astype('category')

In [36]:
df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20692840 entries, 0 to 20692839
Data columns (total 10 columns):
 #   Column         Dtype              
---  ------         -----              
 0   event_time     datetime64[ns, UTC]
 1   event_type     category           
 2   product_id     int32              
 3   category_id    int64              
 4   category_code  category           
 5   brand          category           
 6   price          float64            
 7   user_id        int32              
 8   user_session   object             
 9   source_file    category           
dtypes: category(4), datetime64[ns, UTC](1), float64(1), int32(2), int64(1), object(1)
memory usage: 2.4 GB
